##Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Flatten
from keras.utils import to_categorical
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import regularizers
from keras.initializers import HeNormal

In [ ]:
tr = pd.read_csv('train.csv')

##Load Train Dataset

In [ ]:
def load_data():
    x_train, x_val, y_train, y_val = train_test_split(
        tr.drop('label', axis=1).values,
        tr['label'].values,
        random_state=31,
        train_size=0.7,
        shuffle=True,
        stratify=tr['label'].values
    )

    return x_train, x_val, y_train, y_val

x_train, x_val, y_train, y_val = load_data()

##Preprocessing

In [ ]:
# Normalize with zero-mean and standardize by 255
mean = np.mean(x_train, axis=0)
x_train = (x_train - mean) / 255
x_val = (x_val - mean) / 255

##One Hot Encoding

In [ ]:
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)

##Model

In [ ]:
def cosine_annealing(epoch, lr):
    max_epochs = 50
    return 0.0005 * (1 + math.cos(math.pi * epoch / max_epochs)) / 2

callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    LearningRateScheduler(cosine_annealing)
]

In [ ]:
model = Sequential([
    Dense(512, input_shape=(3072,), kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.01),

    Dense(256, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.01),

    Dense(128, kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.01),

    Dense(10, activation='softmax')
])

model.compile(optimizer=Nadam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.05,
    callbacks=callbacks
)

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step - accuracy: 0.2324 - loss: 2.4050 - val_accuracy: 0.3794 - val_loss: 1.8970 - learning_rate: 5.0000e-04
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 50ms/step - accuracy: 0.3343 - loss: 2.0438 - val_accuracy: 0.4254 - val_loss: 1.8078 - learning_rate: 4.9951e-04
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.3756 - loss: 1.9241 - val_accuracy: 0.4325 - val_loss: 1.7542 - learning_rate: 4.9803e-04
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 23s 54ms/step - accuracy: 0.3986 - loss: 1.8588 - val_accuracy: 0.4437 - val_loss: 1.7084 - learning_rate: 4.9557e-04
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 49ms/step - accuracy: 0.4152 - loss: 1.8054 - val_accuracy: 0.4563 - val_loss: 1.6745 - learning_rate: 4.9215e-04
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - accuracy: 0.4331 - loss: 1.7692 - val_accuracy: 0.4603 - val_loss: 1.6545 - learning_rate: 4.8776e-04
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 38s 52ms/ste

##Validation

In [ ]:
loss, accuracy = model.evaluate(x_val, y_val)
print("Validation Accuracy:", accuracy)

338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5555 - loss: 1.4449
Validation Accuracy: 0.556851863861084


##Evaluation

In [ ]:
ts = pd.read_csv('test.csv')
test = ts.iloc[:, 1:]
index = ts.iloc[:, 0]
test = (test - mean) / 255

Pred_labels = model.predict(test)
predicted_classes = np.argmax(Pred_labels, axis=1)
df = pd.DataFrame({'id': index, 'label': predicted_classes})
df.to_csv('AROOJ-198-A4.csv', index=False)

750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step
